In [68]:
# Set to false if you are not running
# this notebook in Google Colaboratory
run_on_colab = True

In [69]:
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
!pip install music21;

In [ ]:
if(run_on_colab):
  from google.colab import files
  files.upload()

Saving midi_files.zip to midi_files.zip


In [71]:
!unzip midi_files.zip;

Archive:  midi_files.zip
replace midi_files/chopin/chpn-p15_format0.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: midi_files/chopin/chpn-p15_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn-p15_format0.mid  
  inflating: midi_files/chopin/chpn_op35_1_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn_op35_1_format0.mid  
  inflating: midi_files/chopin/chpn-p5_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn-p5_format0.mid  
  inflating: midi_files/chopin/chpn-p19_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn-p19_format0.mid  
  inflating: midi_files/chopin/.DS_Store  
  inflating: __MACOSX/midi_files/chopin/._.DS_Store  
  inflating: midi_files/chopin/chpn_op10_e12_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn_op10_e12_format0.mid  
  inflating: midi_files/chopin/chpn-p9_format0.mid  
  inflating: __MACOSX/midi_files/chopin/._chpn-p9_format0.mid  
  inflating: midi_files/chopin/chpn_op25_e2_format0.mid  
  infl

In [72]:
# Importing dependencies
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [73]:
file = "midi_files/chopin/chpn-p9_format0.mid"
midi = converter.parse(file)
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.chord.Chord B3 E2> 0.0
<music21.chord.Chord E3 G#3> 0.0
<music21.note.Note B> 1/3
<music21.chord.Chord E3 G#3> 2/3
<music21.note.Note B> 1.0
<music21.chord.Chord E-3 F#3> 1.0
<music21.note.Note B> 1.0
<music21.note.Note B> 4/3
<music21.chord.Chord E-3 F#3> 5/3
<music21.note.Note B> 1.75


In [74]:
notes = []
for i,file in enumerate(glob.glob("midi_files/chopin/*.mid")):
  midi = converter.parse(file)
  print('\r', 'Parsing file ', i, " ",file, end='')
  notes_to_parse = None
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
    if isinstance(element, note.Note):
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      notes.append('.'.join(str(n) for n in element.normalOrder))
with open('notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

 Parsing file  50   midi_files/chopin/chpn-p10_format0.mid

In [75]:
# Count different possible outputs
n_vocab = (len(set(notes)))
n_vocab

456

In [76]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length] # Size sequence_length
  sequence_out = notes[i + sequence_length]  # Size 1
  # Map pitches of sequence_in to integers
  network_input.append([note_to_int[char] for char in sequence_in])
  # Map integer of sequence_out to an integer
  network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [77]:
network_input.shape

(55535, 100, 1)

In [78]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

In [79]:
model = create_network(network_input,n_vocab)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100, 512)          2099200   
_________________________________________________________________
dropout_10 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 512)               2099200   
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)              

In [80]:
# In case we want to use previously trained weights
weights = ""
if(len(weights)>0): model.load_weights(weights)

In [81]:
filepath = "/content/drive/My Drive/Train_02/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, 
          callbacks=callbacks_list)

Epoch 1/200
868/868 [==============================] - 71s 81ms/step - loss: 4.9312
Epoch 2/200
868/868 [==============================] - 70s 81ms/step - loss: 4.9092
Epoch 3/200
868/868 [==============================] - 70s 81ms/step - loss: 4.9592
Epoch 4/200
868/868 [==============================] - 70s 80ms/step - loss: 4.9151
Epoch 5/200
868/868 [==============================] - 70s 81ms/step - loss: 4.9452
Epoch 6/200
868/868 [==============================] - 70s 81ms/step - loss: 4.8485
Epoch 7/200
868/868 [==============================] - 70s 81ms/step - loss: 4.8460
Epoch 8/200
868/868 [==============================] - 70s 81ms/step - loss: 4.7704
Epoch 9/200
868/868 [==============================] - 70s 81ms/step - loss: 4.7161
Epoch 10/200
868/868 [==============================] - 70s 81ms/step - loss: 4.6764
Epoch 11/200
868/868 [==============================] - 70s 81ms/step - loss: 4.6140
Epoch 12/200
868/868 [==============================] - 70s 81ms/step - lo

_-----------------------------------------_


In [83]:
# In case we want to use other previously trained weights
weights = "/content/drive/My Drive/Train_02/94-0.5857.h5"
if(len(weights)>0): model.load_weights(weights)

In [84]:
# Generate network input again
network_input = []
output = []
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i + sequence_length]
  sequence_out = notes[i + sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

In [85]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for i,note_index in enumerate(range(500)):
  prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
  prediction_input = prediction_input / float(n_vocab)
  prediction = model.predict(prediction_input, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  prediction_output.append(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

 Predicted  499   F5

In [86]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='Salida_02.mid')

'Salida_02.mid'